In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataset import pca_data_loader
from tqdm import tqdm

In [3]:
# 加载数据
train_loader, test_loader = pca_data_loader()

# 设置设备（如果有 GPU 可用则使用 GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# 定义模型（全连接层：10 输入，10 输出）
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc = nn.Linear(10, 10)

    def forward(self, x):
        return self.fc(x)

In [5]:
# 初始化模型、损失函数和优化器
model = SimpleMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# 训练模型
epochs = 10
with tqdm(total=epochs * len(train_loader), desc="Training Progress", unit='batch') as pbar:
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.update(1)


Training Progress: 100%|██████████| 9380/9380 [00:12<00:00, 735.01batch/s]


In [7]:
# 在测试集上评估模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 74.50%
